In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')
"""
参考:
https://qiita.com/miettal/items/0d322aacaab9ddb2a4fd
"""

pref_code2name = {
   1: "北海道",
   2: "青森県", 3: "岩手県", 4: "宮城県", 5: "秋田県", 6: "山形県", 7: "福島県",
   8: "茨城県", 9: "栃木県", 10: "群馬県", 11: "埼玉県", 12: "千葉県", 13: "東京都", 14: "神奈川県",
   15: "新潟県", 16: "富山県", 17: "石川県", 18: "福井県", 19: "山梨県", 20: "長野県", 21: "岐阜県", 22: "静岡県", 23: "愛知県",
   24: "三重県", 25: "滋賀県", 26: "京都府", 27: "大阪府", 28: "兵庫県", 29: "奈良県", 30: "和歌山県",
   31: "鳥取県", 32: "島根県", 33: "岡山県", 34: "広島県", 35: "山口県",
   36: "徳島県", 37: "香川県", 38: "愛媛県", 39: "高知県",
   40: "福岡県", 41: "佐賀県", 42: "長崎県", 43: "熊本県", 44: "大分県", 45: "宮崎県", 46: "鹿児島県",
   47: "沖縄県",}
#keyとitemの反対
pref_name2code = dict([(v, k) for (k, v) in pref_code2name.items()])

df_jpn = gpd.read_file('data/gadm36_JPN_1.shp')
df_jpn['code'] = df_jpn['NL_NAME_1'].map(pref_name2code.get)
#print('df_jpn.shape ',df_jpn.shape)
df_jpn.plot()
plt.savefig('img/japan_map.png')
plt.clf()

df_vaccine = pd.read_csv('data/vaccine.csv',header = 0)
first_day = df_vaccine[:1]['date'].values[0]
last_days = df_vaccine[-1:]['date'].values[0]
print("from " + first_day + " to " + last_days)
df_population = pd.read_csv('data/population_done.csv',header = 0)
#print(df_population.head())


from 2021-04-12 to 2021-08-23


In [2]:
df_vaccine = df_vaccine.groupby(['date','prefecture'], as_index=False)['count'].sum().fillna(0)
dates = df_vaccine['date'].unique()

In [4]:
cumsum_df = df_vaccine[df_vaccine['date'] == dates[-1]].reset_index(drop = True)
cumsum_df = df_jpn.merge(cumsum_df, left_on='code', right_on='prefecture', how='left')
cumsum_df['count'] = 0

for date in dates:
    df = df_vaccine[df_vaccine['date'] == date].reset_index(drop = True)
    df = df_jpn.merge(df, left_on='code', right_on='prefecture', how='left')
    df['count'] = df['count'].fillna(0)
    df['count'] += cumsum_df['count']
    cumsum_df = df
    
    df = df.merge(df_population, left_on='code', right_on='prefecture_code', how='left')
    df['vaccine_rate'] = df['count'] / df['population'] * 100
    df.plot(column='vaccine_rate', vmin=0, vmax=60.0, cmap='rainbow', edgecolors='black', legend=True)
    plt.title(date)
    plt.savefig('img/animation/' + date + 'japan_vaccine_map.png')
    plt.clf()

In [8]:
#df['vaccine_rate'].max()

53.82497264596944

## 複数の画像からアニメーションを作成

In [9]:
from PIL import Image
import glob
 
# GIFアニメーションを作成
def create_gif(path_list ,output_file = "out.gif"):
    imgs = []                                                 
 
    for path in path_list:
        img = Image.open(path)                   
        imgs.append(img)                                 
    
    #最後だけ少し長くする
    for _ in range(30):
        imgs.append(img)  
        
    
    imgs[0].save(output_file,
                 save_all=True, append_images=imgs[1:], optimize=False, duration=70, loop=0)
    

create_gif(path_list = sorted(glob.glob('img/animation/*.png')))

# 日本全体

In [14]:
japan_population = df_population['population'].sum()
df_japan_all = df_vaccine.groupby('date')['count'].sum().fillna(0) / japan_population 
df_japan_all = df_japan_all.cumsum()

In [15]:
df_japan_all

date
2021-04-12    0.000038
2021-04-13    0.000072
2021-04-14    0.000111
2021-04-15    0.000183
2021-04-16    0.000274
                ...   
2021-07-17    0.293724
2021-07-18    0.297405
2021-07-19    0.299926
2021-07-20    0.303210
2021-07-21    0.305547
Name: count, Length: 101, dtype: float64